# 🧠 Clasificación de la Enfermedad de Alzheimer
Integrantes
- Diego Alexander Hernández Silvestre - 21270
- Linda Inés Jiménez Vides - 21169
- Mario Antonio Guerra Morales - 21008
- Kristopher Javier Alvarado López - 21188

In [5]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import random
import shutil

#### 📊 Balanceo de Data

In [2]:
# Definir las transformaciones que aplicarás
augmentations = transforms.Compose([
    transforms.RandomRotation(degrees=10),  # Rotación de ±10 grados
    transforms.RandomAffine(degrees=0, translate=(0.02, 0.02)),  # Desplazamiento horizontal/vertical 2%
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.92, 1.08)),  # Zoom hasta 8%
    transforms.ToTensor(),
])

# Clase personalizada para cargar imágenes desde carpetas
class DementiaDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_list = os.listdir(image_dir)
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_list[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

# Directorios de las carpetas de imágenes
folders = {
    "MildDemented": "data/train/MildDemented",
    "ModerateDemented": "data/train/ModerateDemented",
    "NonDemented": "data/train/NonDemented",
    "VeryMildDemented": "data/train/VeryMildDemented"
}

target_size = 3200  # Número objetivo de imágenes por clase

# Función para aplicar augmentación y guardar nuevas imágenes
def augment_and_save(dataset, save_dir, current_size, target_size):
    counter = current_size
    loader = DataLoader(dataset, batch_size=1, shuffle=True)
    
    while counter < target_size:
        for batch in loader:
            augmented_img = transforms.ToPILImage()(batch[0])  # Convertir tensor a imagen PIL
            augmented_img.save(os.path.join(save_dir, f'augmented_{counter}.jpg'))  # Guardar imagen
            counter += 1
            if counter >= target_size:
                break

# Iterar sobre cada carpeta
for label, folder in folders.items():
    current_images = os.listdir(folder)
    current_size = len(current_images)
    
    if current_size < target_size:
        print(f"Aplicando augmentación en {label}. Tamaño actual: {current_size}.")
        
        # Cargar el dataset de la clase actual
        dataset = DementiaDataset(image_dir=folder, transform=augmentations)
        
        # Aumentar imágenes y guardar
        augment_and_save(dataset, folder, current_size, target_size)
        
    else:
        print(f"No se necesita augmentación en {label}. Tamaño actual: {current_size}.")


Aplicando augmentación en MildDemented. Tamaño actual: 717.
Aplicando augmentación en ModerateDemented. Tamaño actual: 52.
Aplicando augmentación en NonDemented. Tamaño actual: 2560.
Aplicando augmentación en VeryMildDemented. Tamaño actual: 1792.


In [3]:
for label, folder in folders.items():
    current_images = os.listdir(folder)
    current_size = len(current_images)
    print(f"Clase: {label}. Tamaño actual: {current_size}.")

Clase: MildDemented. Tamaño actual: 3200.
Clase: ModerateDemented. Tamaño actual: 3200.
Clase: NonDemented. Tamaño actual: 3200.
Clase: VeryMildDemented. Tamaño actual: 3200.


#### 🏋🏽‍♀️ División entrenamiento y validación

In [25]:
# Directorios donde se guardarán las imágenes divididas
train_folder = "data/train/"

train_dir = "new_data/train/"
val_dir = "new_data/validation/"

# Crear los directorios de train y validation si no existen
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Contadores para el total de imágenes en train y validation
total_train_images = 0
total_val_images = 0

# Función para dividir y copiar las imágenes de train en train/validation
def split_train_validation(train_folder, label, train_dir, val_dir, split_ratio=0.7):
    global total_train_images, total_val_images
    images = os.listdir(os.path.join(train_folder, label))
    random.shuffle(images)
    
    # Calcular cuántas imágenes irán a train y cuántas a validation
    split_index = int(len(images) * split_ratio)
    train_images = images[:split_index]
    val_images = images[split_index:]
    
    # Actualizar los contadores
    total_train_images += len(train_images)
    total_val_images += len(val_images)
    
    # Crear carpetas de train y validation para la clase actual
    os.makedirs(os.path.join(train_dir, label), exist_ok=True)
    os.makedirs(os.path.join(val_dir, label), exist_ok=True)
    
    # Copiar imágenes de train
    for img in train_images:
        shutil.copy(os.path.join(train_folder, label, img), os.path.join(train_dir, label, img))
    
    # Copiar imágenes de validation
    for img in val_images:
        shutil.copy(os.path.join(train_folder, label, img), os.path.join(val_dir, label, img))

# Iterar sobre las carpetas de cada clase dentro de train
for label in os.listdir(train_folder):
    split_train_validation(train_folder, label, train_dir, val_dir, split_ratio=0.7)

# Imprimir el total de imágenes en train y validation
print(f"Total de imágenes en train: {total_train_images}")
print(f"Total de imágenes en validation: {total_val_images}")
print("División de imágenes de train en train/validation completada.")

Total de imágenes en train: 8960
Total de imágenes en validation: 3840
División de imágenes de train en train/validation completada.


#### Arquitectura del Modelo

In [26]:
# Configuración del dispositivo (GPU si está disponible)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformaciones (Aumento de datos y normalización)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionar las imágenes
    transforms.RandomHorizontalFlip(),  # Voltear horizontalmente
    transforms.RandomRotation(10),  # Rotar aleatoriamente hasta 10 grados
    transforms.ToTensor(),  # Convertir a tensor (esto convierte automáticamente a float y escala a [0, 1])
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalización estándar
])

In [27]:
# Cargar los datasets de entrenamiento y validación
train_data = datasets.ImageFolder('new_data/train', transform=transform)
val_data = datasets.ImageFolder('new_data/validation', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

In [28]:
# Definición del modelo CNN basado en la arquitectura del artículo
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # Entrada 224x224x3 -> Salida 224x224x32
        self.pool = nn.MaxPool2d(2, 2)  # Reduce 2x cada dimensión: 112x112x32 después de la 1ra convolución
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # Entrada 112x112x32 -> Salida 112x112x64
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # Entrada 56x56x64 -> Salida 56x56x128
        self.fc1 = nn.Linear(128 * 28 * 28, 512)  # Capa totalmente conectada, entrada 128*28*28 = 100352, salida 512
        self.fc2 = nn.Linear(512, 4)  # Capa de salida, 4 clases para la clasificación
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Conv1 + MaxPool
        x = self.pool(F.relu(self.conv2(x)))  # Conv2 + MaxPool
        x = self.pool(F.relu(self.conv3(x)))  # Conv3 + MaxPool
        x = x.view(-1, 128 * 28 * 28)  # Aplanar para la capa totalmente conectada
        x = F.relu(self.fc1(x))  # FC1
        x = self.dropout(x)  # Dropout
        x = self.fc2(x)  # FC2 - Salida final
        return x

#### Entrenamiento del Modelo

In [38]:
# Inicializar el modelo
model = CNNModel().to(device)

# Usar CrossEntropyLoss, que combina softmax y entropía cruzada
criterion = nn.CrossEntropyLoss()

# Definir el optimizador
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Entrenamiento del modelo
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f"Empieza el entrenamiento para la época {epoch+1} \n")
        model.train()  # Modo de entrenamiento
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()  # Limpiar gradientes previos
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()  # Calcular gradientes
            optimizer.step()  # Actualizar pesos

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print("Pérdida por época calculada.\n")

        # Validación
        model.eval()  # Modo de evaluación (sin actualización de gradientes)
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)

                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        print("Pérdida por validación calculada.\n")
        val_loss = val_loss / len(val_loader.dataset)
        accuracy = correct / total

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {accuracy:.4f}')

    print('Entrenamiento completo.')

In [39]:
# Entrenar el modelo
train_model(model, train_loader, val_loader, criterion, optimizer)

# Guardar el modelo
torch.save(model.state_dict(), 'model.pth')

Empieza el entrenamiento para la época 1 

Pérdida por época calculada.

Pérdida por validación calculada.

Epoch [1/10], Loss: 1.0241, Val Loss: 0.6928, Val Accuracy: 0.6936
Empieza el entrenamiento para la época 2 

Pérdida por época calculada.

Pérdida por validación calculada.

Epoch [2/10], Loss: 0.6910, Val Loss: 0.6064, Val Accuracy: 0.7091
Empieza el entrenamiento para la época 3 

Pérdida por época calculada.

Pérdida por validación calculada.

Epoch [3/10], Loss: 0.6056, Val Loss: 0.5231, Val Accuracy: 0.7615
Empieza el entrenamiento para la época 4 

Pérdida por época calculada.

Pérdida por validación calculada.

Epoch [4/10], Loss: 0.5416, Val Loss: 0.4639, Val Accuracy: 0.7892
Empieza el entrenamiento para la época 5 

Pérdida por época calculada.

Pérdida por validación calculada.

Epoch [5/10], Loss: 0.5041, Val Loss: 0.4133, Val Accuracy: 0.8245
Empieza el entrenamiento para la época 6 

Pérdida por época calculada.

Pérdida por validación calculada.

Epoch [6/10], Los

In [40]:
# Evaluar el modelo en los datos de validación
def evaluate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    accuracy = correct / total
    print(f'Precisión en el conjunto de validación: {accuracy * 100:.2f}%')

# Evaluar el modelo
evaluate_model(model, val_loader)

Precisión en el conjunto de validación: 90.50%
